<a href="https://colab.research.google.com/github/shiva500/Nepsongs/blob/main/axe_songs_gpt_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [13]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/shiva500/Nepsongs/main/input.txt


--2023-06-07 18:02:32--  https://raw.githubusercontent.com/shiva500/Nepsongs/main/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8369 (8.2K) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   8.17K  --.-KB/s    in 0s      

2023-06-07 18:02:32 (46.8 MB/s) - ‘input.txt’ saved [8369/8369]



In [15]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [16]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3256


In [17]:
# let's look at the first 1000 characters
print(text[:1000])

देखे तिमिलाइ हिजो हे रति सपनी मा
बिपानी मा भेटे को थियान डन्डा परी मा
देखे तिमिलाइ हिजो हे रति सपनी मा
बिपानी मा भेटे को थियान डन्डा परी मा
हो..डांडा परी मा छौत्राई मा-२
चिया बारी मा – २
छाउबन्दी चोली रा टिमरो रातो रिबाना
ढाका सादी मा मस्केरा हिन्द, चोरौ यो मन
उकाली रा ओराली गर्दाइ पन्धेरी मा
चोरी मुतु सुतक्का मान को माझेरी मा
मा ढेरी मा – २
हो...डाँडा परी मा छौत्रै मा-२
चिया बारी मा – २

देउराली को भञ्ज्यागा मै
बार पिपल को छहरी मा प्रीति गानसौ ला
कान्छी तिमिलै डोली मा छडै पञ्चे बाजा मा
लैजाउं ला राखौ ला तिमिलाइ मुतु मेरो मा
मुतु माझा मा – २
हो...डाँडा परी मा छौत्रै मा-२
चिया बारी मा – २

देखे तिमिलाइ हिजो हे रति सपनी मा
बिपानी मा भेटे को थियान डन्डा परी मा
देखे तिमिलाइ हिजो हे रति सपनी मा
बिपानी मा भेटे को थियान डन्डा परी मा
हो..डांडा परी मा छौत्राई मा-२
चिया बारी मा – २
डण्डा परी मा छौत्राई मा-२
चिया बारी मा

आउता चिठी को सहारा ले
टिमरो मान जितना चाहन्ना मा
आउता चिठी को सहारा ले
टिमरो मान जितना चाहन्ना मा
माया कागज मा हुन्न
यो द्वि आत्मा को मेला हो
माया कागज मा हुन्न
यो द्वि आत्मा को

In [18]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 ,-.ँंअआइईउएओकखगघङचछजझञटठडढणतथदधनपफबभमयरलवशसहािीुृेैोौ्२–…
59


In [20]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("देखे तिमिलाइ हिजो"))
print(decode(encode("देखे तिमिलाइ हिजो")))

[31, 51, 15, 51, 1, 29, 47, 38, 47, 41, 46, 9, 1, 45, 47, 21, 53]
देखे तिमिलाइ हिजो


In [21]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([3256]) torch.int64
tensor([31, 51, 15, 51,  1, 29, 47, 38, 47, 41, 46,  9,  1, 45, 47, 21, 53,  1,
        45, 51,  1, 40, 29, 47,  1, 44, 34, 33, 48,  1, 38, 46,  0, 36, 47, 34,
        46, 33, 48,  1, 38, 46,  1, 37, 51, 24, 51,  1, 14, 53,  1, 30, 47, 39,
        46, 33,  1, 26, 33, 55, 26, 46,  1, 34, 40, 48,  1, 38, 46,  0, 31, 51,
        15, 51,  1, 29, 47, 38, 47, 41, 46,  9,  1, 45, 47, 21, 53,  1, 45, 51,
         1, 40, 29, 47,  1, 44, 34, 33, 48,  1, 38, 46,  0, 36, 47, 34, 46, 33,
        48,  1, 38, 46,  1, 37, 51, 24, 51,  1, 14, 53,  1, 30, 47, 39, 46, 33,
         1, 26, 33, 55, 26, 46,  1, 34, 40, 48,  1, 38, 46,  0, 45, 53,  4,  4,
        26, 46,  6, 26, 46,  1, 34, 40, 48,  1, 38, 46,  1, 20, 54, 29, 55, 40,
        46, 10,  1, 38, 46,  3, 56,  0, 19, 47, 39, 46,  1, 36, 46, 40, 48,  1,
        38, 46,  1, 57,  1, 56,  0, 20, 46, 11, 36, 33, 55, 31, 48,  1, 19, 53,
        41, 48,  1, 40, 46,  1, 24, 47, 38, 40, 53,  1, 40, 46, 29, 53,  1, 40,
        4

In [31]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [32]:
block_size = 8
train_data[:block_size+1]

tensor([31, 51, 15, 51,  1, 29, 47, 38, 47])

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([31]) the target: 51
when input is tensor([31, 51]) the target: 15
when input is tensor([31, 51, 15]) the target: 51
when input is tensor([31, 51, 15, 51]) the target: 1
when input is tensor([31, 51, 15, 51,  1]) the target: 29
when input is tensor([31, 51, 15, 51,  1, 29]) the target: 47
when input is tensor([31, 51, 15, 51,  1, 29, 47]) the target: 38
when input is tensor([31, 51, 15, 51,  1, 29, 47, 38]) the target: 47


In [33]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[44, 55, 14, 47,  6, 31, 46,  2],
        [33, 48,  1, 38, 46,  1, 37, 51],
        [47, 38, 47, 41, 46,  9,  1, 45],
        [ 1, 33, 46, 19, 55, 39, 53,  1]])
targets:
torch.Size([4, 8])
tensor([[55, 14, 47,  6, 31, 46,  2,  1],
        [48,  1, 38, 46,  1, 37, 51, 24],
        [38, 47, 41, 46,  9,  1, 45, 47],
        [33, 46, 19, 55, 39, 53,  1, 44]])
----
when input is [44] the target: 55
when input is [44, 55] the target: 14
when input is [44, 55, 14] the target: 47
when input is [44, 55, 14, 47] the target: 6
when input is [44, 55, 14, 47, 6] the target: 31
when input is [44, 55, 14, 47, 6, 31] the target: 46
when input is [44, 55, 14, 47, 6, 31, 46] the target: 2
when input is [44, 55, 14, 47, 6, 31, 46, 2] the target: 1
when input is [33] the target: 48
when input is [33, 48] the target: 1
when input is [33, 48, 1] the target: 38
when input is [33, 48, 1, 38] the target: 46
when input is [33, 48, 1, 38, 46] the target: 1
when input is [33, 48

In [26]:
print(xb) # our input to the transformer

tensor([[44, 55, 14, 47,  6, 31, 46,  2],
        [33, 48,  1, 38, 46,  1, 37, 51],
        [47, 38, 47, 41, 46,  9,  1, 45],
        [ 1, 33, 46, 19, 55, 39, 53,  1]])


In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 59])
tensor(4.6603, grad_fn=<NllLossBackward0>)

घहैृओृलँघलिभ२ुौ–मतीरथैख
चठहभृउईइ र
वझतइयुलरलधपिै.चकडखहिीोए-अ जढंझेौ ृु,उलश
दौलज२आंभडथ
गँ.ुेटु…जध–ोँण


In [34]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [38]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


1.9976385831832886


In [39]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


माउते मराँडाकोजिमा ली २
आउबुका कान्ड मेउत्रो मरो पालै हो पो मिमिपा

मे थिसो देटिला

यान्नारी मा लकेरी को तन्फु झे मान्छौंगजो मा
टिजसो भोलछी मि बोला-लाइ नेर त मा बो को सायो…
तिमिता म्दे हे
बि हिठी – केखेला भन्न्न्मिपराया उतुझिलकिमराल्कान्सा मसंड जा सो तृ दी मझ्को हितमाइ ल्या थिलु
त्ध्दे, झायाउंडा खा च्ंडै मिमा
चा गज झ्नाद्या रवि रा
हन
या ला मिया, लान्र्है को
ब मा पञ्वि चोरोजसाइ २
रोला जण्र्यालो, किपानी थिसो…
हा मानार्.. रा त्दी ले ले झि मा
भा बि.डा माडारो – हन्न्न

सा उंगरी क – परो मानजा कोला मि 


## The mathematical trick in self-attention

In [40]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [41]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [42]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [43]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [44]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [45]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [46]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [47]:
k.var()

tensor(1.0449)

In [48]:
q.var()

tensor(1.0700)

In [49]:
wei.var()

tensor(1.0918)

In [50]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [51]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [52]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [53]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [54]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [55]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.208955 M parameters
step 0: train loss 4.2458, val loss 4.2221
step 100: train loss 2.2483, val loss 2.3148
step 200: train loss 1.9629, val loss 2.1141
step 300: train loss 1.5092, val loss 1.7468
step 400: train loss 1.0769, val loss 1.4338
step 500: train loss 0.7570, val loss 1.1601
step 600: train loss 0.5088, val loss 1.0257
step 700: train loss 0.3729, val loss 0.8782
step 800: train loss 0.3139, val loss 0.8402
step 900: train loss 0.2698, val loss 0.7307
step 1000: train loss 0.2462, val loss 0.7979
step 1100: train loss 0.2365, val loss 0.7916
step 1200: train loss 0.2224, val loss 0.8002
step 1300: train loss 0.2152, val loss 0.7701
step 1400: train loss 0.2047, val loss 0.7707
step 1500: train loss 0.2049, val loss 0.7780
step 1600: train loss 0.1968, val loss 0.7934
step 1700: train loss 0.1910, val loss 0.7447
step 1800: train loss 0.1915, val loss 0.8340
step 1900: train loss 0.1944, val loss 0.8569
step 2000: train loss 0.1900, val loss 0.8174
step 2100: train loss 0.